In [47]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account 

In [48]:
# Nodes runing with POW
web3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))


# Loading Mnemonic EV and (set this mnemonic as an environment variable, and include the one you generated as a fallback using)
mnemonic = os.getenv('MNEMONIC', "save amazing snack goose market glad glad fee vintage beauty enable frequent")
print(mnemonic)

save amazing snack goose market glad glad fee vintage beauty enable frequent


In [49]:
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys

In [50]:
#Setting dictionary of coins to be used in the wallet
coins = {"eth", "btc-test", "btc"}
numderive = 3

In [51]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [52]:
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc"
"cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt"


In [53]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "1Hk26uv3aSo2WmKNPbeUGjNWPFDBLU2waW",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KzQ3o74J1R3MFuzvRNQWjckRz3AYZHgggJRbs3P7TH1ZoXJqKety",
            "pubkey": "0399c9a8f6933934bfe811fc524c0e8684f971819398a0581f20ddf6371db0bb18",
            "pubkeyhash": "b7a4d19972d97725ebe1d980ac875f4b41322ec5",
            "xprv": "xprvA4EtnVRNTKMCodewrh6ipRZg96w7tyJMntxXmtPitFkMn7Dwe6BRfeC2bnL8DRs96stL1Kn752eZJqfFX1Fs25uaRxnUSSShU91qzeg5gWK",
            "xpub": "xpub6HEFBzxGHguW27jQxidjBZWQh8mcJS2DA7t8aGoLSbHLeuZ6BdVgDSWWT5kbpKZxGjjA8qDfgYCyKgBeau5Qp48c5KC9dTaYjWd3jLfF8MG"
        },
        {
            "address": "16LsmsW5nshaMWz4XXE9qZjE1HHEEF61Ma",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L3PRGxRUzyYGDEnWsdtyunpa5DCfYcgKfBSCCXGEHofUyxYj7zc6",
            "pubkey": "03412c487c7a81706519d3236215f717b9134a39e0447e1a70de08fd65d83ab7e1",
            "pubkeyhash":

In [54]:
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [55]:
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)

btc-test
cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt
eth
0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc


In [56]:
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD>


In [57]:
def create_tx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global trx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)]) 

In [58]:
def send_tx(coin, account, recipient, amount):
    """call create_trx, sign the transaction, then send it to the designated network"""
    if coin == "eth": 
        trx_eth = create_trx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_trx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

In [72]:
create_tx(BTCTEST,btc_acc,"miZgMxdGzSxCTpWazfD2KqhewoUvcQ6CC1", 0.00001)

'{"unspents":[{"amount":326459,"confirmations":0,"script":"76a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac","txid":"d306b881a357826f3af534a0d55ed06576d0a367289839d3d74748e886be1aba","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["miZgMxdGzSxCTpWazfD2KqhewoUvcQ6CC1",1000],["mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD",302407]]}'

In [73]:
send_tx(BTCTEST,btc_acc,'miZgMxdGzSxCTpWazfD2KqhewoUvcQ6CC1',0.00001)

'0100000001ba1abe86e84847d7d339982867a3d07665d05ed5a034f53a6f8257a381b806d3010000006a47304402201a26242bd6bd1b09f83e191ff6cca5db35ea1ab467a8dfe40d0a516fccf8af7602202b95d3ea99dfccd54428324f244a642bc1c96fd561c624b2431cef3022dedc48012102dcb7a11c7172966e58679c0cb207bfb95d119cc55f9fb318d0a522aaff5b9577ffffffff02e8030000000000001976a914216c53486c93fa265d93a5ea85b6d3499b136d9188ac479d0400000000001976a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac00000000'

In [61]:
from web3.middleware import geth_poa_middleware

web3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [62]:
from web3 import Web3, HTTPProvider

In [63]:
#connecting to HTTP with address pk
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x5619d0cce54919511eb9f3b678de07bd346044acee0bdd6cdf4497ba45e03f81"))


In [64]:
#checking the Block Number
w3.eth.blockNumber

9

In [65]:
# double check if  I am connected to blockchain. 
w3.isConnected()

True

In [66]:
w3.eth.getBalance("0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e")

0

In [68]:
create_tx(ETH,eth_acc,"0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e", 1000)

{'to': '0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e',
 'from': '0x02194a55DDA1B029F70220D53f099F38c629ab36',
 'value': 1000,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}

In [69]:
send_tx(ETH, eth_acc,"0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e", 1000)

ValueError: {'message': "sender doesn't have enough funds to send tx. The upfront cost is: 420000000001000 and the sender's account only has: 0", 'code': -32000, 'data': {'stack': "Error: sender doesn't have enough funds to send tx. The upfront cost is: 420000000001000 and the sender's account only has: 0\n    at VM.<anonymous> (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\lib\\runTx.ts:114:11)\n    at step (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\dist\\runTx.js:33:23)\n    at Object.next (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\dist\\runTx.js:14:53)\n    at fulfilled (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\res

In [71]:
web3.eth.getBalance("0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e")

ConnectionError: HTTPConnectionPool(host='127.0.0.1.8545', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000259D4485588>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))